# Settings

In [1]:
# Load Reticulate function
Sys.setenv(RETICULATE_PYTHON="/home/luca/anaconda3/envs/reticulate/bin/python")
library(reticulate)
reticulate::use_python("/home/luca/anaconda3/envs/reticulate/bin/python")
reticulate::use_condaenv("/home/luca/anaconda3/envs/reticulate")
reticulate::py_module_available(module='anndata') #needs to be TRUE
reticulate::import('anndata') #good to make sure this doesn't error
reticulate::py_module_available(module='leidenalg') #needs to be TRUE
reticulate::import('leidenalg') #good to make sure this doesn't error

[1] TRUE

Module(anndata)

[1] TRUE

Module(leidenalg)

In [2]:
## Patch for annotations in R4.1
# BiocManager::install("Bioconductor/GenomeInfoDb",lib = "/home/luca/R/x86_64-pc-linux-gnu-library/4.1",force = TRUE)
# library(GenomeInfoDb,lib.loc="/home/luca/R/x86_64-pc-linux-gnu-library/4.1")

In [3]:
# Load packages
pacman::p_load(dplyr, stringr, data.table, tidyr, data.table, Matrix, tidyverse,
               hdf5r, Seurat, Signac,harmony, knitr, SoupX, fgsea,
               logr, parallel, DESeq2,
               ggplot2, ggpubr, ggrepel, ggbreak, gridExtra, patchwork, grid, ggh4x)

In [5]:
# Set options
options(stringsAsFactors = FALSE)
warnLevel <- getOption('warn')
options(warn = -1)
opts_chunk$set(tidy=TRUE)

In [6]:
# Set directories
assets.dir = "/nfs/lab/projects/COVID_mouse/assets/"
reference.dir = "/nfs/lab/projects/COVID_mouse/assets/BoneMarrowReference.rds"
snRNAseq.dir = "/nfs/lab/projects/COVID_mouse/seurat/5_clean.map/snRNAseq_mrg.clean.scTyped.rds"

reference.map.dir = "/nfs/lab/projects/COVID_mouse/5_clean.map/"
counts.dir = paste0(reference.map.dir, "Downstream_Files/RNA/perdonor/COUNTS/")
DESEQ.dir = paste0(reference.map.dir, "DESEQ2/")
GSEA.res.dir = paste0(reference.map.dir, "DESEQ2/GSEA/")


# DESeq2

In [7]:
# create metadata
samples = c("1_GFP1", "2_GFP2", "3_GFP3",
           "4_RBD1", "5_RBD2", "6_RBD3",
           "10_G1C1", "11_G1C2", "12_G1C3")

# R doesn't like columns that have a number, and will add an "X", will do the same here
samples = paste0("X", samples)
conditions = c("GFP", "GFP", "GFP",
             "RBD", "RBD", "RBD",
             "G1C", "G1C", "G1C")
meta = data.frame(donor = samples, condition = conditions)
meta

donor,condition
<chr>,<chr>
X1_GFP1,GFP
X2_GFP2,GFP
X3_GFP3,GFP
X4_RBD1,RBD
X5_RBD2,RBD
X6_RBD3,RBD
X10_G1C1,G1C
X11_G1C2,G1C
X12_G1C3,G1C


In [8]:
#Pseudobulk matrices directory
dir <- counts.dir
#Create outdir for results
outdir <- DESEQ.dir
# dir.create(outdir)
#Get list of pseudobulk files
files <- list.files(dir, pattern='_perdonor.gex_SoupX.RNA.counts')
#Cut off file suffices to get celltype names
cells <- gsub('_perdonor.gex_SoupX.RNA.counts','', files) 

In [9]:
files
cells

[1] "B_cell_perdonor.gex_SoupX.RNA.counts"         
 [2] "Dendritic_cells_perdonor.gex_SoupX.RNA.counts"
 [3] "Eo_Baso_prog._perdonor.gex_SoupX.RNA.counts"  
 [4] "Ery_Mk_prog._perdonor.gex_SoupX.RNA.counts"   
 [5] "Erythroblasts_perdonor.gex_SoupX.RNA.counts"  
 [6] "Gran_Mono_prog._perdonor.gex_SoupX.RNA.counts"
 [7] "large_pre-B._perdonor.gex_SoupX.RNA.counts"   
 [8] "LMPPs_perdonor.gex_SoupX.RNA.counts"          
 [9] "Mk_prog._perdonor.gex_SoupX.RNA.counts"       
[10] "Mono_prog._perdonor.gex_SoupX.RNA.counts"     
[11] "Monocytes_perdonor.gex_SoupX.RNA.counts"      
[12] "Neutro_prog._perdonor.gex_SoupX.RNA.counts"   
[13] "Neutrophils_perdonor.gex_SoupX.RNA.counts"    
[14] "NK_cells_perdonor.gex_SoupX.RNA.counts"       
[15] "pro-B_perdonor.gex_SoupX.RNA.counts"          
[16] "small_pre-B._perdonor.gex_SoupX.RNA.counts"   
[17] "T_cells_perdonor.gex_SoupX.RNA.counts"

[1] "B_cell"          "Dendritic_cells" "Eo_Baso_prog."   "Ery_Mk_prog."   
 [5] "Erythroblasts"   "Gran_Mono_prog." "large_pre-B."    "LMPPs"          
 [9] "Mk_prog."        "Mono_prog."      "Monocytes"       "Neutro_prog."   
[13] "Neutrophils"     "NK_cells"        "pro-B"           "small_pre-B."   
[17] "T_cells"

In [10]:
# List your contrasts  
conditions.1 = c("G1C", "RBD", "G1C") # disease
conditions.2 = c("GFP", "GFP", "RBD") # control
contrasts = c(paste0(conditions.1, "_vs_", conditions.2))

# Write your formual
deseq.formula <- as.formula('~  condition') # last variable is the testing variable

In [37]:
deseq.stats = NULL

for (i in seq_along(files)) {
    file.use = files[i]
    cell.use = cells[i]
    print(paste0("Analyzing: ", cell.use))
    raw_counts <- read.table(paste0(dir, file.use), header = TRUE, row.names = 1)
    # Filter for only samples with any counts in the cell type
    raw_counts = raw_counts[, (colSums(raw_counts != 0) > 0)]
    message("Donors detectected with celltype: ", ncol(raw_counts))
    meta_cell <- subset(meta, donor %in% colnames(raw_counts))
    rownames(meta_cell) = meta_cell$donor
    message("subsetting metadata accordingly - Check that value is equal to above: ", nrow(meta_cell))
    # Ensure that the column names of raw_counts are in the same order as the library identifiers in meta_cell
    meta_cell <- meta_cell[match(colnames(raw_counts), rownames(meta_cell)), ]
    
    for (c.i in seq_along(contrasts)) {
        contrast.use = contrasts[c.i]
        condition.1 = conditions.1[c.i]
        condition.2 = conditions.2[c.i]
        message("  - ", contrast.use)

        # Create vectors of samples for each condition
        samples.condition.1 <- meta_cell %>% 
                               dplyr::filter(condition %in% condition.1) %>% 
                               pull(donor)
        samples.condition.2 <- meta_cell %>% 
                               dplyr::filter(condition %in% condition.2) %>% 
                               pull(donor)
        
        # Subset meta_cell for the current contrast
        meta_cell.use <- meta_cell %>% 
                     dplyr::filter(donor %in% c(samples.condition.1, samples.condition.2)) 
        rownames(meta_cell.use) <- meta_cell.use$donor
        
        # Subset raw_counts for the donors in meta_cell
        raw_counts_subset <- raw_counts[, colnames(raw_counts) %in% rownames(meta_cell.use)]

        # Ensure that the column names of raw_counts are in the same order as the library identifiers in meta_cell
        meta_cell.use <- meta_cell.use[match(colnames(raw_counts_subset), rownames(meta_cell.use)), ]
        
        # Check if enough samples are present for both conditions
        if (length(unique(meta_cell$condition)) >= 2) {
            deseq.stats.tmp <- data.frame()
            
            # Light pre-filtering
            counts_filtered <- raw_counts_subset[rowSums(raw_counts_subset) >= 10,]

            # Run DESeq2
            dds <- DESeqDataSetFromMatrix(countData = round(counts_filtered),
                                          colData = meta_cell.use, 
                                          design = deseq.formula)
            dds$condition <- relevel(dds$condition, ref = condition.2)
            dds <- DESeq(dds)
            dds.results <- results(dds)
            dds.results <- na.omit(dds.results[order(dds.results$padj), ])

            # Write DESeq results to file
            write.table(dds.results, 
                        file = paste0(outdir, cell.use, "_", contrast.use, ".dds.res"), 
                        sep = '\t', quote = FALSE)
            
            # Collect summary statistics
            deseq.stats.tmp <- data.frame(
                celltype = cell.use,
                contrast = contrast.use,
                up_pval = sum(dds.results$log2FoldChange > 0 & dds.results$pvalue < 0.05),
                down_pval = sum(dds.results$log2FoldChange < 0 & dds.results$pvalue < 0.05),
                up_padj = sum(dds.results$log2FoldChange > 0 & dds.results$padj < 0.1),
                down_padj = sum(dds.results$log2FoldChange < 0 & dds.results$padj < 0.1))
                deseq.stats <- rbind(deseq.stats, deseq.stats.tmp)

                           # Make Volcano plot - Padj
                            res = as.data.frame(dds.results)
                            res$SYMBOL = rownames(res)
                            res = res %>%
                            dplyr::filter(!is.na(SYMBOL)) %>%
                            dplyr::filter(padj < 0.1) %>%
                            dplyr::group_by(sign(log2FoldChange)) %>%
                            dplyr::mutate(rank = abs(log2FoldChange) * -log10(padj)) %>%
                            dplyr::top_n(50, rank) %>%
                            dplyr::mutate(delabel = SYMBOL) %>%
                            ungroup() %>%
                            dplyr::select(delabel, SYMBOL) %>%
                            right_join(res) %>%
                            dplyr::mutate(delabel = if_else(is.na(delabel), "", as.character(delabel))) 

                            # Add a new column for color based on significance and direction
                            res$changes <- ifelse(res$log2FoldChange > 0 & res$padj < 0.1, "Up-regulated",
                                                ifelse(res$log2FoldChange < 0 & res$padj < 0.1, "Down-regulated", "NS"))
                            # Get the number of upregulated and downregulated genes
                            n_up <- sum(res$changes == "Up-regulated")
                            n_down <- sum(res$changes == "Down-regulated")

                            # Modify log2FoldChange values to be max x
                            res$log2FoldChange[res$log2FoldChange > 10] <- 10
                            res$log2FoldChange[res$log2FoldChange < -10] <- -10
                            # Modify padj values to be max x 
                            res$padj[res$padj < 10^-200] <- 10^-200
                    
                            plot = ggplot(res, aes(x=log2FoldChange, y=-log10(padj), label=delabel, color=changes)) +
                                theme_bw() +
                                labs(y="-log10(padj)", x="log2FoldChange", 
                                   title=paste(cell.use, " - ",
                                               condition.1, "Vs", condition.2, "\n",
                                              "Up: ", n_up, "-",
                                               "Down: ", n_down)) +
                                geom_point() +
                                scale_color_manual(values=c("blue", "grey", "red")) + 
                                theme(axis.text=element_text(size=16), axis.title=element_text(size=16, face="bold"),
                                    axis.text.x=element_text(), plot.title=element_text(size=18, face="bold", hjust=0.5)) +
                                scale_x_continuous(breaks=c(-10, -5, 0, 5, 10),
                                            labels=c("< -10", "-5", "0", "5", "> 10"), lim = c(-10, 10))+
                                scale_y_continuous(breaks=c(0, 20, 40, 60, 80, 100, 200),
                                            labels=c("0", "20", "40", "60", "80", "100", ">200"))+
                                geom_text_repel(label=res$delabel, size=3.5, max.overlaps=10,
                                              point.padding=0, min.segment.length=0,
                                              max.time=20, max.iter=1e5, box.padding=0.3)
                    
                            ggsave(filename = paste0(outdir, cell.use, "_", contrast.use, ".volcanoPlot.pdf"),
                                   height = 10, width = 10,
                                   plot = plot, device = "pdf")
        } else {
            message("Not enough samples for contrast: ", contrast.use)
        }
    }
}

[1] "Analyzing: B_cell"


Donors detectected with celltype: 9

subsetting metadata accordingly - Check that value is equal to above: 9

  - G1C_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - RBD_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - G1C_vs_RBD

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`


[1] "Analyzing: Dendritic_cells"


Donors detectected with celltype: 9

subsetting metadata accordingly - Check that value is equal to above: 9

  - G1C_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - RBD_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - G1C_vs_RBD

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`


[1] "Analyzing: Eo_Baso_prog."


...

Donors detectected with celltype: 9

subsetting metadata accordingly - Check that value is equal to above: 9

  - G1C_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - RBD_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - G1C_vs_RBD

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`


[1] "Analyzing: small_pre-B."


Donors detectected with celltype: 9

subsetting metadata accordingly - Check that value is equal to above: 9

  - G1C_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - RBD_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - G1C_vs_RBD

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`


[1] "Analyzing: T_cells"


Donors detectected with celltype: 9

subsetting metadata accordingly - Check that value is equal to above: 9

  - G1C_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - RBD_vs_GFP

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`
  - G1C_vs_RBD

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Joining with `by = join_by(SYMBOL)`


In [38]:
deseq.stats

celltype,contrast,up_pval,down_pval,up_padj,down_padj
<chr>,<chr>,<int>,<int>,<int>,<int>
B_cell,G1C_vs_GFP,247,348,18,39
B_cell,RBD_vs_GFP,84,199,0,7
B_cell,G1C_vs_RBD,339,172,35,8
Dendritic_cells,G1C_vs_GFP,459,320,66,30
Dendritic_cells,RBD_vs_GFP,644,812,166,304
Dendritic_cells,G1C_vs_RBD,1321,1177,945,694
Eo_Baso_prog.,G1C_vs_GFP,101,111,3,3
Eo_Baso_prog.,RBD_vs_GFP,30,23,0,0
Eo_Baso_prog.,G1C_vs_RBD,75,78,4,1


In [46]:
write.table(deseq.stats, 
            file = paste0(outdir, "DESEQ_summary.txt"), 
            sep = '\t', quote = FALSE)

# FGSEA

In [19]:
GSEA.dir = "/nfs/lab/Luca/Assets/Gene.Sets/mouse_categories/"

In [35]:
setwd(GSEA.dir)
gene.sets.ls = dir()

gene.set.db = NULL
for (i in seq_along(gene.sets.ls)){
    gene.set = gene.sets.ls[i]
    message("Processing ", gene.set)
    # Read in the GMT as a table, to get the name of the term
    gene.set.use = read.table(gene.set, sep = "\t", fill = TRUE)
    # Subset to just the first column
    gene.set.use = as.data.frame(gene.set.use[,1])
    colnames(gene.set.use) = "pathway"
    gene.set.use$term = as.factor(str_split_fixed(gene.set.use$pathway, pattern = "_", n = 2)[,2])
    gene.set.use$set = as.factor(str_split_fixed(gene.set.use$pathway, pattern = "_", n = 2)[,1])
            tmp = str_split_fixed(gene.set, pattern = "_", n = 2)[,2]
    gene.set.use$set.family = as.factor(str_split_fixed(tmp, pattern = "\\.", n = 2)[,1])
    gene.set.use$set.superfamily = as.factor(str_split_fixed(gene.set, pattern = "_", n = 2)[,1])

    gene.set.db = rbind(gene.set.db, gene.set.use)
}

gene.set.db$set.superfamily = factor(gene.set.db$set.superfamily, 
                                     levels = c("GO"))
gene.set.db$set.family = factor(gene.set.db$set.family, 
                                     levels = c("BP", "CC", "MF",
                                                "CP"))

Processing GO_BP.v2023.2.Mm.symbols.gmt.txt

Processing GO_CC.v2023.2.Mm.symbols.gmt.txt

Processing GO_CP.v2023.2.Mm.symbols.gmt.txt

Processing GO_MF.v2023.2.Mm.symbols.gmt.txt



In [40]:
# Load the pathways into a named list
GMT <- gmtPathways("/nfs/lab/Luca/Assets/Gene.Sets/msigdb.v2023.2.Mm.symbols.gmt")

In [41]:
#Pseudobulk matrices directory
dir <- DESEQ.dir
#Create outdir for results
outdir <- GSEA.res.dir
# dir.create(outdir)
#Get list of pseudobulk files
files <- list.files(dir, pattern='.dds.res')
files

[1] "B_cell_G1C_vs_GFP.dds.res"          "B_cell_G1C_vs_RBD.dds.res"         
 [3] "B_cell_RBD_vs_GFP.dds.res"          "Dendritic_cells_G1C_vs_GFP.dds.res"
 [5] "Dendritic_cells_G1C_vs_RBD.dds.res" "Dendritic_cells_RBD_vs_GFP.dds.res"
 [7] "Eo_Baso_prog._G1C_vs_GFP.dds.res"   "Eo_Baso_prog._G1C_vs_RBD.dds.res"  
 [9] "Eo_Baso_prog._RBD_vs_GFP.dds.res"   "Ery_Mk_prog._G1C_vs_GFP.dds.res"   
[11] "Ery_Mk_prog._G1C_vs_RBD.dds.res"    "Ery_Mk_prog._RBD_vs_GFP.dds.res"   
[13] "Erythroblasts_G1C_vs_GFP.dds.res"   "Erythroblasts_G1C_vs_RBD.dds.res"  
[15] "Erythroblasts_RBD_vs_GFP.dds.res"   "Gran_Mono_prog._G1C_vs_GFP.dds.res"
[17] "Gran_Mono_prog._G1C_vs_RBD.dds.res" "Gran_Mono_prog._RBD_vs_GFP.dds.res"
[19] "large_pre-B._G1C_vs_GFP.dds.res"    "large_pre-B._G1C_vs_RBD.dds.res"   
[21] "large_pre-B._RBD_vs_GFP.dds.res"    "LMPPs_G1C_vs_GFP.dds.res"          
[23] "LMPPs_G1C_vs_RBD.dds.res"           "LMPPs_RBD_vs_GFP.dds.res"          
[25] "Mk_prog._G1C_vs_GFP.dds.res"        "Mk_prog._G1C_vs_RBD.dds.res"       
[27] "Mk_prog._RBD_vs_GFP.dds.res"        "Mono_prog._G1C_vs_GFP.dds.res"     
[29] "Mono_prog._G1C_vs_RBD.dds.res"      "Mono_prog._RBD_vs_GFP.dds.res"     
[31] "Monocytes_G1C_vs_GFP.dds.res"       "Monocytes_G1C_vs_RBD.dds.res"      
[33] "Monocytes_RBD_vs_GFP.dds.res"       "Neutro_prog._G1C_vs_GFP.dds.res"   
[35] "Neutro_prog._G1C_vs_RBD.dds.res"    "Neutro_prog._RBD_vs_GFP.dds.res"   
[37] "Neutrophils_G1C_vs_GFP.dds.res"     "Neutrophils_G1C_vs_RBD.dds.res"    
[39] "Neutrophils_RBD_vs_GFP.dds.res"     "NK_cells_G1C_vs_GFP.dds.res"       
[41] "NK_cells_G1C_vs_RBD.dds.res"        "NK_cells_RBD_vs_GFP.dds.res"       
[43] "pro-B_G1C_vs_GFP.dds.res"           "pro-B_G1C_vs_RBD.dds.res"          
[45] "pro-B_RBD_vs_GFP.dds.res"           "small_pre-B._G1C_vs_GFP.dds.res"   
[47] "small_pre-B._G1C_vs_RBD.dds.res"    "small_pre-B._RBD_vs_GFP.dds.res"   
[49] "T_cells_G1C_vs_GFP.dds.res"         "T_cells_G1C_vs_RBD.dds.res"        
[51] "T_cells_RBD_vs_GFP.dds.res"

In [45]:
# setup
tresh = 0.1
set.family.Keep = c("CP")

In [47]:
set.seed(999)

for (c.i in seq_along(contrasts)){
    contrast.use = contrasts[c.i]
        for (i in seq_along(files)){
            # load table
            file.use = files[i]
            cell.use = str_split_fixed(file.use, paste0("_", contrast.use), n = 2)[,1]
            De.analysis = paste0(cell.use, "_", contrast.use)
            print(paste0("Analyzing: ", De.analysis))
            res = read.table(paste0(dir,file.use), sep = '\t')

            # Formatting for FGSEA
            res$rank = res$stat
            res = data.frame("SYMBOL" = rownames(res),
                             "stat" = res$rank)
            res = res[!grepl(pattern = "NA", x = res$SYMBOL),]
            ranks <- deframe(res)
            ranks = sort(ranks, decreasing=TRUE)

            tryCatch({
            message("Running FGSEA")
            fgseaRes <- fgseaMultilevel(pathways=GMT,
                                        stats=ranks,
                                        minSize  = 10, 
                                        maxSize  = 500)
            message("Number of total enriched terms: ", nrow(fgseaRes))
            fgseaRes.tresh = fgseaRes[fgseaRes$padj < tresh,]
            message("Number of significant terms: ", nrow(fgseaRes.tresh))
            # Add categories
            fgseaRes.tresh = merge(fgseaRes.tresh, gene.set.db, by = "pathway")
            message("Double check that it's the same number of terms: ", nrow(fgseaRes.tresh))
            # Subset for GO families of interest
            fgseaRes.tresh.use = fgseaRes.tresh[fgseaRes.tresh$set.family %in% set.family.Keep, ]
            message("Number of significant terms: ", nrow(fgseaRes.tresh.use))
            # Collapse terms 
            collapsedPathways <- collapsePathways(fgseaRes.tresh.use[order(pval)][padj < tresh], 
                                                  GMT, ranks)
            mainPathways <- fgseaRes.tresh.use[pathway %in% collapsedPathways$mainPathways][
                                     order(-NES), pathway]
            fgseaResMain <- fgseaRes.tresh.use[match(mainPathways, pathway)]
            # Save them
            file = paste(outdir, "GSEA.CP_", De.analysis, ".res", sep = "")
            fwrite(fgseaRes.tresh.use, file = file, sep = "\t")
            # print plot
            plot = ggplot(fgseaResMain, aes(x = reorder(term, NES), y = NES)) +
              geom_col(aes(fill= NES>0)) +
              theme_bw()+
              labs(x="", y="Normalized Enrichment Score",
                   title= paste(De.analysis,
                                " \n Gene Ontologies")) + 
                theme(axis.text=element_text(size=10), axis.title=element_text(size=16, face="bold"),
                axis.text.x=element_text(), plot.title=element_text(size=16, face="bold", hjust=0.5)) +
              coord_flip() +
            facet_nested(rows = vars(set.family, set), drop = TRUE, scales = "free", space = "free")
            ggsave(filename = paste(outdir, "GSEA.CP_", De.analysis, ".pdf", sep = ""),
                                    height = 16, width = 20,
                                    plot = plot, device = "pdf")
                }, error = function(e) {
            message("Error or no results found for ", De.analysis, ": ", e$message)
        })
    }
}

[1] "Analyzing: B_cell_G1C_vs_GFP"


Running FGSEA

Number of total enriched terms: 9783

Number of significant terms: 1733

Double check that it's the same number of terms: 1025

Number of significant terms: 194



[1] "Analyzing: B_cell_G1C_vs_RBD.dds.res_G1C_vs_GFP"


Running FGSEA

Number of total enriched terms: 9577

Number of significant terms: 1923

Double check that it's the same number of terms: 1155

Number of significant terms: 215



[1] "Analyzing: B_cell_RBD_vs_GFP.dds.res_G1C_vs_GFP"


...

Running FGSEA

Number of total enriched terms: 6961

Number of significant terms: 448

Double check that it's the same number of terms: 276

Number of significant terms: 49



[1] "Analyzing: T_cells_G1C_vs_RBD"


Running FGSEA

Number of total enriched terms: 7425

Number of significant terms: 692

Double check that it's the same number of terms: 433

Number of significant terms: 96



[1] "Analyzing: T_cells_RBD_vs_GFP.dds.res_G1C_vs_RBD"


Running FGSEA

Number of total enriched terms: 8721

Number of significant terms: 985

Double check that it's the same number of terms: 750

Number of significant terms: 128



In [48]:
# setup
tresh = 0.1
set.family.Keep = c("BP")

In [ ]:
set.seed(999)

for (c.i in seq_along(contrasts)){
    contrast.use = contrasts[c.i]
        for (i in seq_along(files)){
            # load table
            file.use = files[i]
            cell.use = str_split_fixed(file.use, paste0("_", contrast.use), n = 2)[,1]
            De.analysis = paste0(cell.use, "_", contrast.use)
            print(paste0("Analyzing: ", De.analysis))
            res = read.table(paste0(dir,file.use), sep = '\t')

            # Formatting for FGSEA
            res$rank = res$stat
            res = data.frame("SYMBOL" = rownames(res),
                             "stat" = res$rank)
            res = res[!grepl(pattern = "NA", x = res$SYMBOL),]
            ranks <- deframe(res)
            ranks = sort(ranks, decreasing=TRUE)

            tryCatch({
            message("Running FGSEA")
            fgseaRes <- fgseaMultilevel(pathways=GMT,
                                        stats=ranks,
                                        minSize  = 10, 
                                        maxSize  = 500)
            message("Number of total enriched terms: ", nrow(fgseaRes))
            fgseaRes.tresh = fgseaRes[fgseaRes$padj < tresh,]
            message("Number of significant terms: ", nrow(fgseaRes.tresh))
            # Add categories
            fgseaRes.tresh = merge(fgseaRes.tresh, gene.set.db, by = "pathway")
            message("Double check that it's the same number of terms: ", nrow(fgseaRes.tresh))
            # Subset for GO families of interest
            fgseaRes.tresh.use = fgseaRes.tresh[fgseaRes.tresh$set.family %in% set.family.Keep, ]
            message("Number of significant terms: ", nrow(fgseaRes.tresh.use))
            # Collapse terms 
            collapsedPathways <- collapsePathways(fgseaRes.tresh.use[order(pval)][padj < tresh], 
                                                  GMT, ranks)
            mainPathways <- fgseaRes.tresh.use[pathway %in% collapsedPathways$mainPathways][
                                     order(-NES), pathway]
            fgseaResMain <- fgseaRes.tresh.use[match(mainPathways, pathway)]
            # Save them
            file = paste(outdir, "GSEA.BP_", De.analysis, ".res", sep = "")
            fwrite(fgseaRes.tresh.use, file = file, sep = "\t")
            # print plot
            plot = ggplot(fgseaResMain, aes(x = reorder(term, NES), y = NES)) +
              geom_col(aes(fill= NES>0)) +
              theme_bw()+
              labs(x="", y="Normalized Enrichment Score",
                   title= paste(De.analysis,
                                " \n Gene Ontologies")) + 
                theme(axis.text=element_text(size=10), axis.title=element_text(size=16, face="bold"),
                axis.text.x=element_text(), plot.title=element_text(size=16, face="bold", hjust=0.5)) +
              coord_flip() +
            facet_nested(rows = vars(set.family, set), drop = TRUE, scales = "free", space = "free")
            ggsave(filename = paste(outdir, "GSEA.BP_", De.analysis, ".pdf", sep = ""),
                                    height = 16, width = 20,
                                    plot = plot, device = "pdf")
                }, error = function(e) {
            message("Error or no results found for ", De.analysis, ": ", e$message)
        })
    }
}

[1] "Analyzing: B_cell_G1C_vs_GFP"


Running FGSEA

Number of total enriched terms: 9783

Number of significant terms: 1733

Double check that it's the same number of terms: 1025

Number of significant terms: 634



[1] "Analyzing: B_cell_G1C_vs_RBD.dds.res_G1C_vs_GFP"


Running FGSEA

Number of total enriched terms: 9577

Number of significant terms: 1844

Double check that it's the same number of terms: 1102

Number of significant terms: 673



[1] "Analyzing: B_cell_RBD_vs_GFP.dds.res_G1C_vs_GFP"


...

Running FGSEA

Number of total enriched terms: 8721

Number of significant terms: 969

Double check that it's the same number of terms: 738

Number of significant terms: 504



[1] "Analyzing: B_cell_G1C_vs_GFP.dds.res_G1C_vs_RBD"


Running FGSEA

Number of total enriched terms: 9783

Number of significant terms: 1552

Double check that it's the same number of terms: 906

Number of significant terms: 556



[1] "Analyzing: B_cell_G1C_vs_RBD"


Running FGSEA

Number of total enriched terms: 9577

Number of significant terms: 1860

Double check that it's the same number of terms: 1111

Number of significant terms: 682



In [ ]:
# setup
tresh = 0.1
set.family.Keep = c("CC")

In [ ]:
set.seed(999)

for (c.i in seq_along(contrasts)){
    contrast.use = contrasts[c.i]
        for (i in seq_along(files)){
            # load table
            file.use = files[i]
            cell.use = str_split_fixed(file.use, paste0("_", contrast.use), n = 2)[,1]
            De.analysis = paste0(cell.use, "_", contrast.use)
            print(paste0("Analyzing: ", De.analysis))
            res = read.table(paste0(dir,file.use), sep = '\t')

            # Formatting for FGSEA
            res$rank = res$stat
            res = data.frame("SYMBOL" = rownames(res),
                             "stat" = res$rank)
            res = res[!grepl(pattern = "NA", x = res$SYMBOL),]
            ranks <- deframe(res)
            ranks = sort(ranks, decreasing=TRUE)

            tryCatch({
            message("Running FGSEA")
            fgseaRes <- fgseaMultilevel(pathways=GMT,
                                        stats=ranks,
                                        minSize  = 10, 
                                        maxSize  = 500)
            message("Number of total enriched terms: ", nrow(fgseaRes))
            fgseaRes.tresh = fgseaRes[fgseaRes$padj < tresh,]
            message("Number of significant terms: ", nrow(fgseaRes.tresh))
            # Add categories
            fgseaRes.tresh = merge(fgseaRes.tresh, gene.set.db, by = "pathway")
            message("Double check that it's the same number of terms: ", nrow(fgseaRes.tresh))
            # Subset for GO families of interest
            fgseaRes.tresh.use = fgseaRes.tresh[fgseaRes.tresh$set.family %in% set.family.Keep, ]
            message("Number of significant terms: ", nrow(fgseaRes.tresh.use))
            # Collapse terms 
            collapsedPathways <- collapsePathways(fgseaRes.tresh.use[order(pval)][padj < tresh], 
                                                  GMT, ranks)
            mainPathways <- fgseaRes.tresh.use[pathway %in% collapsedPathways$mainPathways][
                                     order(-NES), pathway]
            fgseaResMain <- fgseaRes.tresh.use[match(mainPathways, pathway)]
            # Save them
            file = paste(outdir, "GSEA.CC_", De.analysis, ".res", sep = "")
            fwrite(fgseaRes.tresh.use, file = file, sep = "\t")
            # print plot
            plot = ggplot(fgseaResMain, aes(x = reorder(term, NES), y = NES)) +
              geom_col(aes(fill= NES>0)) +
              theme_bw()+
              labs(x="", y="Normalized Enrichment Score",
                   title= paste(De.analysis,
                                " \n Gene Ontologies")) + 
                theme(axis.text=element_text(size=10), axis.title=element_text(size=16, face="bold"),
                axis.text.x=element_text(), plot.title=element_text(size=16, face="bold", hjust=0.5)) +
              coord_flip() +
            facet_nested(rows = vars(set.family, set), drop = TRUE, scales = "free", space = "free")
            ggsave(filename = paste(outdir, "GSEA.CC_", De.analysis, ".pdf", sep = ""),
                                    height = 16, width = 20,
                                    plot = plot, device = "pdf")
                }, error = function(e) {
            message("Error or no results found for ", De.analysis, ": ", e$message)
        })
    }
}